Представим, что международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров и обратилось для этого к вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

In [8]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.cluster import MeanShift

In [33]:
np.linalg.norm([1, 1])

1.4142135623730951

In [29]:
addresses = (
    (33.751277, -118.188740),
    (25.867736, -80.324116),
    (51.503016, -0.075479),
    (52.378894, 4.885084),
    (39.366487, 117.036146),
    (-33.868457, 151.205134)
)

In [3]:
df = pd.read_csv("checkins.dat", sep="|")
df.columns = list(map(lambda x: x.strip(), df.columns))
df.drop(axis=0, index=1021966, inplace=True)
df = df[df.latitude != '                   ']

In [4]:
df.drop("id user_id venue_id created_at".split(), axis=1, inplace=True)

In [5]:
df.latitude = df.latitude.astype(float);
df.longitude = df.longitude.astype(float);

In [9]:
df.shape
df.head()

(396634, 2)

,latitude,longitude
1,38.895112,-77.036366
3,33.800745,-84.410520
7,45.523452,-122.676207
9,40.764462,-111.904565
10,33.448377,-112.074037


In [10]:
train_df = df[:100000]
X_train = train_df.values

In [11]:
X_train.shape

(100000, 2)

In [12]:
mean_shift = MeanShift(bandwidth=0.1)

In [13]:
%%time
mean_shift.fit(X_train)

Wall time: 3min 9s


MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, min_bin_freq=1,
          n_jobs=None, seeds=None)

In [17]:
mean_shift.cluster_centers_.shape

(3231, 2)

In [14]:
train_df["clusters"] = mean_shift.predict(X_train)

In [15]:
train_df

,latitude,longitude,clusters
1,38.895112,-77.036366,5
3,33.800745,-84.410520,7
7,45.523452,-122.676207,30
9,40.764462,-111.904565,65
10,33.448377,-112.074037,1
...,...,...,...
233788,33.575000,-117.725556,50
233789,37.629349,-122.400087,4
233793,29.762884,-95.383061,25
233797,32.802955,-96.769923,19


In [18]:
clusters_size = train_df.clusters.value_counts()

In [184]:
clusters = pd.concat(
    (pd.DataFrame(
        mean_shift.cluster_centers_,
        columns="latitude_center longitude_center".split()),
     clusters_size),
    axis=1
)
clusters = clusters[clusters.clusters > 15]

In [185]:
clusters['length'] = float("inf")

In [186]:
clusters.head()

,latitude_center,longitude_center,clusters,length
0,40.717716,-73.991835,12506,inf
1,33.449438,-112.002140,4692,inf
2,33.446380,-111.901888,3994,inf
3,41.878244,-87.629843,3363,inf
4,37.688682,-122.409330,3526,inf


In [183]:
some = np.column_stack(((clusters.latitude_center - lat, clusters.longitude_center - long)))
# hstack not working correct :( 
some

array([[  74.5861734 , -225.19696942],
       [  67.31789505, -263.20727369],
       [  67.31483727, -263.10702156],
       ...,
       [  75.48698875, -239.65070217],
       [  73.1179256 , -228.3872611 ],
       [  72.52723615, -228.0908211 ]])

In [191]:
clusters[["length_", "length"]]

,length_,length
0,44.742571,inf
1,6.193959,inf
2,6.294241,inf
3,31.621097,inf
4,5.772048,inf
...,...,...
684,45.451826,inf
727,45.144291,inf
884,30.766052,inf
1343,41.373571,inf


In [193]:
for i, (lat, long) in enumerate(addresses):
    clusters["length_"] = np.linalg.norm(
        np.column_stack(
            ((clusters.latitude_center - lat, clusters.longitude_center - long))),
        axis=1
    )
    clusters.length = np.min(clusters[["length_", "length"]], axis=1)

clusters.drop("length_", axis=1, inplace=True)

In [180]:
clusters.length = np.min(clusters[names], axis=1)

In [195]:
clusters.head()

,latitude_center,longitude_center,clusters,length
0,40.717716,-73.991835,12506,16.143720
1,33.449438,-112.002140,4692,6.193959
2,33.446380,-111.901888,3994,6.294241
3,41.878244,-87.629843,3363,17.598580
4,37.688682,-122.409330,3526,5.772048


In [197]:
clusters.sort_values("length", axis=0)

,latitude_center,longitude_center,clusters,length
420,-33.860630,151.204776,28,0.007835
370,52.372964,4.892317,31,0.009353
419,25.845672,-80.318891,28,0.022674
58,51.502991,-0.125537,254,0.050058
51,33.809878,-118.148924,281,0.070848
...,...,...,...,...
105,-23.549518,-46.638219,116,59.806393
433,-33.425741,-70.570740,27,60.090305
262,-22.903539,-43.209587,47,61.287238
103,-0.301226,36.522964,120,61.450391


In [202]:
print(*clusters.sort_values("length", axis=0).iloc[0, :2].values)

-33.86063042857143 151.20477592857145
